# Python Beyond Basics (Lesson 8)

> 继承 和 多态

## Simple Inheritence

class SubClass(BaseClass)

* SubClass will want to initialize base classes
* BaseClass initializer will only be called automatically if **subclass intializer is undefined**

In [65]:
class Base:
    
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('Base.f()')

In [66]:
b = Base()

Base initializer


In [67]:
b.f()

Base.f()


In [68]:
class Sub(Base):
    pass

In [69]:
s = Sub()

Base initializer


In [70]:
s.f()

Base.f()


重写 Sub 的 f 方法:

In [71]:
class Sub(Base):
    
    def f(self):
        print('Sub.f()')

In [72]:
s = Sub()

Base initializer


In [73]:
s.f()

Sub.f()


重写 Sub 的 `__init__` 方法:

In [74]:
class Sub(Base):
    
    def __init__(self):
        print('Sub initializer')
    
    def f(self):
        print('Sub.f()')

In [75]:
s = Sub()

Sub initializer


In [76]:
s.f()

Sub.f()


Python的继承：

如果被重写了`__init__()` 不会自动调用 Base 的 `__init()__` ,需要手动 `super().__init__()`

![Python Inheritence](https://i.loli.net/2018/01/10/5a55d32d08fbf.png)


In [77]:
class Sub(Base):
    
    def __init__(self):
        super().__init__()
        print('Sub initializer')
    
    def f(self):
        print('Sub.f()')

In [78]:
s = Sub()

Base initializer
Sub initializer


### SimpleList

In [79]:
class SimpleList:
    
    def __init__(self, items):
        self._items = list(items)
        
    def add(self, item):
        self._items.append(item)
        
    def __getitem__(self, idx):
        return self._items[idx]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return 'SimpleList({!r})'.format(list(self))

In [80]:
class SortedList(SimpleList):
    
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()
        
    def add(self, item):
        super().add(item)
        self.sort()
        
    def __repr__(self):
        return 'SortedList({!r})'.format(list(self))

Tips: 多次 用 super() 去掉用父类的Method(base-class implemention)

In [81]:
sl = SortedList([4, 3, 7, 9])

In [82]:
sl

SortedList([3, 4, 7, 9])

In [83]:
sl.add(0)

In [84]:
sl

SortedList([0, 3, 4, 7, 9])

### Multiple Inheritence

Multiple Inheritence in Python is **not much more complex** than single inheritence


isinstance() 
* determins if an object is a specified type
* Use isinstance() for runtime type checking

In [85]:
 isinstance(3, int)

True

In [86]:
 isinstance('hello', str)

True

In [87]:
isinstance(sl, SortedList)

True

In [88]:
isinstance(sl, SimpleList)

True

In [89]:
x = []
isinstance(x, (list, dict, tuple))

True

In [90]:
# 加入 item 的类型判断

class IntList(SimpleList):
    
    def __init__(self, items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
    
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only support int values')
    
    def add(self, item):
        self._validate(item)
        super().add(item)
        
    def __repr__(self):
        return 'IntList({!r})'.format(list(self))

In [91]:
il = IntList([1, 2, 3])
il.add(5)

In [92]:
il

IntList([1, 2, 3, 5])

In [93]:
il.add('6')

TypeError: IntList only support int values

issubclass() - determines if one type is a subclass of another

In [94]:
issubclass(IntList, SimpleList)

True

In [95]:
issubclass(IntList, SortedList)

False

In [96]:
issubclass(SimpleList, object)

True

In [97]:
class MyInt(int): pass
class MyVerySpecialInt(MyInt): pass

issubclass(MyVerySpecialInt, int)

True

### Multiple Inheritence

> (C++ 支持， Java不支持)

defining a class with more than one base class

class SubClass(Base1, Base2,...)

* SubClass inherits methods of all bases
* Without conflict, name resolve in the obvious way
* Method Resolution Order(MRO) determines name lookup in all cases

In [98]:
class SortedIntList(IntList, SortedList):
    
    # 注意没有定义 add(), __init__(), 调用的时候 遵守 MRO
    
    def __repr__(self):
        return 'SortedIntList({!r})'.format(list(self))

In [99]:
# 不支持 str

SortedIntList([1, 2, '3'])

TypeError: IntList only support int values

In [100]:
sil = SortedIntList([3, 5, 1])
sil

SortedIntList([1, 3, 5])

In [101]:
sil.add('6')

TypeError: IntList only support int values

In [102]:
sil.add(6)
sil

SortedIntList([1, 3, 5, 6])

Multiple Inheritence:

If a class:

A. has multiple base classes

B. defines no initializer

then only the initializer of the first base class is automatically called

In [103]:
class Base1:
    def __init__(self):
        print('Base1.__init__ called')

class Base2:
    def __init__(self):
        print('Base2.__init__ called')
        
class Sub(Base1, Base2):
    pass

In [104]:
s = Sub()

Base1.__init__ called


`__bases__` a tuple of base classes

In [105]:
Sub.__bases__

(__main__.Base1, __main__.Base2)

In [106]:
SortedIntList.__bases__

(__main__.IntList, __main__.SortedList)

In [107]:
IntList.__bases__

(__main__.SimpleList,)

### MRO

* Methods may be defined in multiple places
* MRO is an ordering of the inheritence graph
* it's very simple

mro 返回一个 list，在list 从头开始找method，找到了就直接返回，在前面的就先返回，就这么简单

#### C3 Princile

* Subclasses come before base class
* BaseClass order from class definition is preserved
* First 2 qualities are preserved no matter where you start in the inheritence graph


#### Super()

Given a method resolution order(MRO) and a class C, super() gives you an 
object which resolves methods using only the part of the MRO which comes after C

* super() returns a proxy object which routes method calls
    * Bound Proxy - bound to a specific class or instance (only talk about this)
    * Unbound Proxy - not bound to a class or instance

#### Bound Proxies

* instance-bound
    * super(class, instance-of-class)
    * Python finds MRO for the type of second arg
    * Find the location of the first argument in the MRO
    * Uses everything after that for resolving methods
    
* class-bound
    * super(base-class, derived-class)
    * Python finds MRO for derived-class
    * It then finds base-class in that MRO
    * It take everything after base-class in the MRO, and finds the first class in that sequence with a matching method name
    
#### super()

super()

* if instance method = super(class-of-method, self)
* if class method = super(class-of-method, class)
* super() uses everything after a specific class in a MRO to resolve method calls 

In [112]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [113]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

In [116]:
class A: pass

class B(A): pass

class C(A): pass

# 会出错， A 不能在C前面
class D(B, A, C): pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

In [119]:
# Class Bound

super(SortedList, SortedIntList).add

<function __main__.SimpleList.add>

In [125]:
# Instance Bound

sil = SortedIntList([3, 2, 6])

In [126]:
sil

SortedIntList([2, 3, 6])

In [127]:
super(SortedList, sil).add

<bound method SimpleList.add of SortedIntList([2, 3, 6])>

In [128]:
super(SortedList, sil).add(5)

In [129]:
sil

SortedIntList([2, 3, 6, 5])

#### Both classes uses super() instead of direct base  classes references


![super 调用](https://i.loli.net/2018/01/12/5a5859e1a47ca.png)



## 总结

![总结](https://i.loli.net/2018/01/12/5a585c836bb9c.png)




In [131]:
class A:
    
    def __init__(self):
        print('A init')
        
class B(A):
    
    def __init__(self):
        print('B init before')
        super().__init__()
        print('B init after')

class C(A):
    
    def __init__(self):
        print('C init before')
        super().__init__()
        print('C init after')
        
class D(B, C):
    pass

In [132]:
d = D()

B init before
C init before
A init
C init after
B init after
